# Sources
* Customised implementation of the journal article:
    * pepsickle rapidly and accurately predicts proteasomal cleavage sites for improved neoantigen identification
    * Benjamin R Weeder, Mary A Wood, Ellysia Li, Abhinav Nellore, Reid F Thompson
    * https://academic.oup.com/bioinformatics/article/37/21/3723/6363787
    * https://github.com/pdxgx/pepsickle
    * https://github.com/pdxgx/pepsickle-paper

In [1]:
import os
import sys
import csv
import math
import random
import numpy as np
from time import time
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torchtext.vocab import build_vocab_from_iterator
from torchtext.vocab import vocab

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold

In [2]:
def seed_everything(seed: int):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything(1234)

In [3]:
def read_data(path):
    with open(path, "r") as csvfile:
        train_data = list(csv.reader(csvfile))[1:]  # skip col name
        sents, lbls = [], []
        for s, l in train_data:
            sents.append(s)
            lbls.append(l)
    return sents, lbls

def apply_random_masking(lst):
    """
    Mask a random position as unknown per sequence.
    """
    lst[random.randint(0, 9)] = '*'
    return lst


def regularized_auc(train_auc, dev_auc, threshold=0.0025):
    """
    Returns development AUC if overfitting is below threshold, otherwise 0.
    """
    return dev_auc if (train_auc - dev_auc) < threshold else 0


def save_metrics(*args, path):
    if not os.path.isfile(path):
        with open(path, "w", newline="\n") as f:
            f.write(
                ",".join(
                    [
                        "fold",
                        "epoch",
                        "train_loss",
                        "train_acc",
                        "train_auc",
                        "val_loss",
                        "val_acc",
                        "val_auc",
                    ]
                )
            )
            f.write("\n")
    if args:
        with open(path, "a", newline="\n") as f:
            f.write(",".join([str(arg) for arg in args]))
            f.write("\n")  
            
def trainable_model_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def total_model_params(model):
    return sum(p.numel() for p in model.parameters())

In [4]:
class CleavageDataset(Dataset):
    def __init__(self, seq, lbl):
        self.seq = seq
        self.lbl = lbl

    def __getitem__(self, idx):
        return self.seq[idx], self.lbl[idx]

    def __len__(self):
        return len(self.lbl)


class TrainBatch:
    def __init__(self, batch):
        ordered_batch = list(zip(*batch))
        self.seq = torch.tensor(
            [
                [_features[aa] for aa in apply_random_masking(list(seq))]
                for seq in ordered_batch[0]
            ],
            dtype=torch.float,
        )
        self.lbl = torch.tensor([int(l) for l in ordered_batch[1]], dtype=torch.float)

    def pin_memory(self):
        self.seq = self.seq.pin_memory()
        self.lbl = self.lbl.pin_memory()
        return self


def train_wrapper(batch):
    return TrainBatch(batch)


class EvalBatch:
    def __init__(self, batch):
        ordered_batch = list(zip(*batch))
        self.seq = torch.tensor(
            [
                [_features[aa] for aa in list(seq)]
                for seq in ordered_batch[0]
            ],
            dtype=torch.float,
        )
        self.lbl = torch.tensor([int(l) for l in ordered_batch[1]], dtype=torch.float)

    def pin_memory(self):
        self.seq = self.seq.pin_memory()
        self.lbl = self.lbl.pin_memory()
        return self


def eval_wrapper(batch):
    return EvalBatch(batch)

In [5]:
# model architectures taken from 
# https://github.com/pdxgx/pepsickle-paper/blob/master/scripts/modeling/epitope_based_ensemble_net.py

class SeqNet(nn.Module):
    def __init__(self, hidden_size1, hidden_size2, hidden_size3, dropout):
        super().__init__()
        
        self.dropout = nn.Dropout(dropout)
        
        # input to linear: seq_len * 20
        self.fc1 = nn.Linear(200, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, hidden_size3)
        self.fc4 = nn.Linear(hidden_size3, 1)
        
        self.bn1 = nn.BatchNorm1d(hidden_size1)
        self.bn2 = nn.BatchNorm1d(hidden_size2)
        self.bn3 = nn.BatchNorm1d(hidden_size3)
        
    def forward(self, seq):
        out = self.dropout(F.relu(self.bn1(self.fc1(seq))))
        out = self.dropout(F.relu(self.bn2(self.fc2(out))))
        out = self.dropout(F.relu(self.bn3(self.fc3(out))))
        return self.fc4(out).squeeze()
    
    
class MotifNet(nn.Module):
    def __init__(self, hidden_size1, hidden_size2, dropout):
        super().__init__()
        
        self.dropout = nn.Dropout(dropout)
        
        # conv parameters are fixed due to feature assemply process
        # see dictionary variable _features
        self.conv = nn.Conv1d(
            in_channels=4,
            out_channels=4,
            kernel_size=3,
            groups=4
        )
    
        # input to linear: groups * (seq_len-2)
        self.fc1 = nn.Linear(32, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, 1)
        
        self.bn1 = nn.BatchNorm1d(hidden_size1)
        self.bn2 = nn.BatchNorm1d(hidden_size2)
        
    def forward(self, seq):
        out = self.conv(seq.transpose(1, 2))
        
        out = self.dropout(F.relu(self.bn1(self.fc1(out.view(out.shape[0], -1)))))
        out = self.dropout(F.relu(self.bn2(self.fc2(out))))
        return self.fc3(out).squeeze()

In [6]:
def train(model, loader, criterion, optim, motif=None):
    epoch_loss, num_correct, total = 0, 0, 0
    preds, lbls = [], []
    
    for batch in tqdm(
        loader,
        desc="Train: " if optim is not None else "Eval: ",
        file=sys.stdout,
        unit="batches"
    ):
        seq, lbl = batch.seq, batch.lbl
        seq, lbl = seq.to(device), lbl.to(device)
        
        if motif is not None:        
            scores = model(seq[:, :, 22:])
        else:
            scores = model(seq[:, :, :20].reshape(seq.shape[0], -1))
            
        loss = criterion(scores, lbl)
        
        optim.zero_grad()
        loss.backward()
        optim.step()
        
        epoch_loss += loss.item()
        num_correct += ((scores > 0) == lbl).sum().item()
        total += seq.shape[0]
        preds.extend(scores.detach().tolist())
        lbls.extend(lbl.detach().tolist())
    return epoch_loss / total, num_correct / total, roc_auc_score(lbls, preds)

In [7]:
def evaluate(seq_model, motif_model, loader, criterion):
    num_correct, total = 0, 0
    preds, lbls = [], []
    
    for batch in tqdm(
        loader,
        desc="Train: " if optim is not None else "Eval: ",
        file=sys.stdout,
        unit="batches"
    ):
        seq, lbl = batch.seq, batch.lbl
        seq, lbl = seq.to(device), lbl.to(device)
               
        motif_scores = motif_model(seq[:, :, 22:])
        seq_scores = seq_model(seq[:, :, :20].reshape(seq.shape[0], -1))
        scores = (motif_scores + seq_scores) / 2

        num_correct += ((scores > 0) == lbl).sum().item()
        total += seq.shape[0]
        preds.extend(scores.detach().tolist())
        lbls.extend(lbl.detach().tolist())
    return num_correct / total, roc_auc_score(lbls, preds)

In [8]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 512

# load train+dev set, mix it back as one
train_path = '../../data/c_train.csv'
dev_path = '../../data/c_val.csv'
test_path = '../../data/c_test.csv'

# combine previously split train and dev set
train_seqs, train_lbls = read_data(train_path)
dev_seqs, dev_lbls = read_data(dev_path)

In [9]:
# see https://github.com/pdxgx/pepsickle README for more info
# 'X' is interpreted as the presence of an amino acid sequence with unkonwn identity
# '*' is interpreted as the absence of amino acid context all together
# we will use '*' as a data augmentation technique during training

_features = {
    'A': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,   0,   0,      6.0, 56.15265,   -0.495,  -2.4],
    'C': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,   0,   0,     5.07, 69.61701,    0.081,  -4.7],
    'D': [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,   0,   0,     2.77, 70.04515,    9.573,  -4.5],
    'E': [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,   0,   0,     3.22, 86.35615,    3.173,  -5.2],
    'F': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,   1,   0,     5.48,  119.722,   -0.370,  -4.9],
    'G': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,   0,   0,     5.97, 37.80307,    0.386,  -1.9],
    'H': [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,   1,   0,     7.59, 97.94236,    2.029,  -4.4],
    'I': [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,   0,   0,     6.02, 103.6644,   -0.528,  -6.6],
    'K': [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,   0,   0,     9.74, 102.7783,    2.101,  -7.5],
    'L': [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,   0,   0,     5.98, 102.7545,   -0.342,  -6.3],
    'M': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,   0,   0,     5.74,  103.928,   -0.324,  -6.1],
    'N': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,   0,   0,     5.41, 76.56687,    2.354,  -4.7],
    'P': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,   0,   0,      6.3, 71.24858,   -0.322,  -0.8],
    'Q': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,   0,   0,     5.65, 88.62562,    2.176,  -5.5],
    'R': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,   0,   0,    10.76, 110.5867,    4.383,  -6.9],
    'S': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,   0,   1,     5.68, 55.89516,    0.936,  -4.6],
    'T': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,   0,   1,      5.6,  72.0909,    0.853,  -5.1],
    'V': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,   0,   0,     5.96, 86.28358,   -0.308,  -4.6],
    'W': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,   1,   0,     5.89, 137.5186,    -0.27,  -4.8],
    'Y': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,   1,   1,     5.66, 121.5862,    1.677,  -5.4],
    '*': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,   0,   0,      7.5,      0.0, 1.689157,   0.0],
    'X': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.5, 0.5, 6.008095, 88.55829,   0.6195, -4.845]
}

In [10]:
BATCH_SIZE=512

train_data = CleavageDataset(train_seqs, train_lbls)
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, collate_fn=train_wrapper, pin_memory=True, num_workers=10)
dev_data = CleavageDataset(dev_seqs, dev_lbls)
dev_loader = DataLoader(dev_data, batch_size=BATCH_SIZE, collate_fn=eval_wrapper, pin_memory=True, num_workers=10)

In [11]:
motif_model = MotifNet(128, 64, dropout=0.2).to(device)
seq_model = SeqNet(256, 128, 64, 0.2).to(device)

criterion = nn.BCEWithLogitsLoss()

motif_optimizer = optim.Adam(motif_model.parameters(), lr=1e-3)
seq_optimizer = optim.Adam(seq_model.parameters(), lr=1e-3)

In [12]:
NUM_EPOCHS = 20

start = time()
print("Starting Cross-Validation.")
highest_val_auc = 0

# normal training loop
for epoch in range(1, NUM_EPOCHS + 1):
    seq_model.train()
    motif_model.train()
    seq_train_loss, seq_train_acc, seq_train_auc = train(
        seq_model, train_loader, criterion, seq_optimizer
    )
    motif_train_loss, motif_train_acc, motif_train_auc = train(
        motif_model, train_loader, criterion, motif_optimizer, motif=True,
    )

    seq_model.eval()
    motif_model.eval()
    with torch.no_grad():
        val_acc, val_auc = evaluate(seq_model, motif_model, dev_loader, criterion)


    print(
        f"Seq-Training:   [Epoch {epoch:2d}, Loss: {seq_train_loss:8.6f}, Acc: {seq_train_acc:.4f}, AUC: {seq_train_auc:.4f}]"
    )
    print(
        f"Mot-Training:   [Epoch {epoch:2d}, Loss: {motif_train_loss:8.6f}, Acc: {motif_train_acc:.4f}, AUC: {motif_train_auc:.4f}]"
    )
    print(f"Evaluation: [Epoch {epoch:2d}, Acc: {val_acc:.4f}, AUC: {val_auc:.4f}]")

print("Finished Cross-Validation.")
train_time = (time() - start) / 60
print(f"Cross-Validation took {train_time} minutes.")

Starting Cross-Validation.
Train: 100%|█████████████████████████████████████████████████████| 278/278 [00:00<00:00, 543.34batches/s]
Seq-Training:   [Epoch  1, Loss: 0.000682, Acc: 0.8474, AUC: 0.8380]
Mot-Training:   [Epoch  1, Loss: 0.000737, Acc: 0.8351, AUC: 0.8010]
Evaluation: [Epoch  1, Acc: 0.8569, AUC: 0.8651]
Train: 100%|█████████████████████████████████████████████████████| 278/278 [00:00<00:00, 553.03batches/s]
Seq-Training:   [Epoch  2, Loss: 0.000654, Acc: 0.8541, AUC: 0.8532]
Mot-Training:   [Epoch  2, Loss: 0.000705, Acc: 0.8411, AUC: 0.8230]
Evaluation: [Epoch  2, Acc: 0.8580, AUC: 0.8695]
Train: 100%|█████████████████████████████████████████████████████| 278/278 [00:00<00:00, 550.89batches/s]
Seq-Training:   [Epoch  3, Loss: 0.000648, Acc: 0.8560, AUC: 0.8567]
Mot-Training:   [Epoch  3, Loss: 0.000698, Acc: 0.8429, AUC: 0.8277]
Evaluation: [Epoch  3, Acc: 0.8588, AUC: 0.8713]
Train: 100%|█████████████████████████████████████████████████████| 278/278 [00:00<00:00, 536.0